## 4-1 Multivariate Linear Regression
<img src="mlr.png" width = "70%">

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 1. set data

In [2]:
x_train = torch.FloatTensor([[73, 80, 75],
                                             [93, 88, 93],
                                             [89, 91, 90],
                                             [96, 98, 100],
                                             [73, 66, 70]]) # 5x3
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]]) # 5x1

### 2. weight, bias 초기화

In [3]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

### 3. set optimizer 

In [4]:
optimizer = optim.SGD([W, b], lr=1e-5)

### 4. training

In [5]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # 한번에 계산 -> 간결, 빠름
    hypothesis = x_train.matmul(W) + b 

    # cost 계산, 기존 lr과 공식 동일
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    #.detach(), item()
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.], grad_fn=<SqueezeBackward0>) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605],
       grad_fn=<SqueezeBackward0>) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821],
       grad_fn=<SqueezeBackward0>) Cost: 2915.712646
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097],
       grad_fn=<SqueezeBackward0>) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307],
       grad_fn=<SqueezeBackward0>) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891],
       grad_fn=<SqueezeBackward0>) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812],
       grad_fn=<SqueezeBackward0>) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.680

### use nn.Module

In [6]:
# nn.Moudule 상속
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear = nn.Linear(3, 1) # 입력차원 : 3, 출력 차원 : 1
    
    # hypothesis 계산
    def forward(self, x):
        return self.linear(x)

In [7]:
x_train = torch.FloatTensor([[73, 80, 75],
                                             [93, 88, 93],
                                             [89, 91, 90],
                                             [96, 98, 100],
                                             [73, 66, 70]]) # 5x3
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]]) # 5x1

In [8]:
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정, model.parameters() 중 일부만 업데이트 선택가능
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 52329.074219
Epoch    1/20 Cost: 16403.128906
Epoch    2/20 Cost: 5142.250000
Epoch    3/20 Cost: 1612.563843
Epoch    4/20 Cost: 506.193451
Epoch    5/20 Cost: 159.405289
Epoch    6/20 Cost: 50.705193
Epoch    7/20 Cost: 16.632938
Epoch    8/20 Cost: 5.952977
Epoch    9/20 Cost: 2.605037
Epoch   10/20 Cost: 1.555364
Epoch   11/20 Cost: 1.226085
Epoch   12/20 Cost: 1.122596
Epoch   13/20 Cost: 1.089863
Epoch   14/20 Cost: 1.079336
Epoch   15/20 Cost: 1.075760
Epoch   16/20 Cost: 1.074353
Epoch   17/20 Cost: 1.073645
Epoch   18/20 Cost: 1.073140
Epoch   19/20 Cost: 1.072717
Epoch   20/20 Cost: 1.072298


### 4-2 Loading Data
    - mini batch gradient descent
   <img src="mini.png">

In [9]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
        
    # 데이터셋의 총 데이터 수
    def __len__(self):
        return len(self.x_data)
    
    # 인덱스(idx)에 상응하는 입출력 데이터 반환
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y

dataset = CustomDataset()

In [10]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size=2, # mini batch 크기, 통상적으로 2의 제곱수 설정
    shuffle=True, # epoch마다 데이터를 섞어 학습 순서를 바꾼다.
)

In [11]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
                epoch, nb_epochs, batch_idx+1, len(dataloader),
                cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 1.277818
Epoch    0/20 Batch 2/3 Cost: 1.137947
Epoch    0/20 Batch 3/3 Cost: 0.798413
Epoch    1/20 Batch 1/3 Cost: 1.667681
Epoch    1/20 Batch 2/3 Cost: 0.812515
Epoch    1/20 Batch 3/3 Cost: 2.335343
Epoch    2/20 Batch 1/3 Cost: 0.877132
Epoch    2/20 Batch 2/3 Cost: 1.753293
Epoch    2/20 Batch 3/3 Cost: 0.236129
Epoch    3/20 Batch 1/3 Cost: 1.899709
Epoch    3/20 Batch 2/3 Cost: 1.188036
Epoch    3/20 Batch 3/3 Cost: 1.110950
Epoch    4/20 Batch 1/3 Cost: 1.932301
Epoch    4/20 Batch 2/3 Cost: 0.592520
Epoch    4/20 Batch 3/3 Cost: 2.513117
Epoch    5/20 Batch 1/3 Cost: 0.479776
Epoch    5/20 Batch 2/3 Cost: 1.794568
Epoch    5/20 Batch 3/3 Cost: 1.017071
Epoch    6/20 Batch 1/3 Cost: 2.457901
Epoch    6/20 Batch 2/3 Cost: 1.723419
Epoch    6/20 Batch 3/3 Cost: 0.014909
Epoch    7/20 Batch 1/3 Cost: 0.963804
Epoch    7/20 Batch 2/3 Cost: 1.530631
Epoch    7/20 Batch 3/3 Cost: 1.251554
Epoch    8/20 Batch 1/3 Cost: 1.207062
Epoch    8/20 Batch 2/3 C